In [144]:
import numpy as np

import pandas as pd

from sklearn.model_selection import KFold

from sklearn.metrics import mean_squared_error

import xgboost as xgb

import math

In [145]:
INPUT_PATH = '../input/'
OUTPUT_PATH = '../output/'


def load_data():
    train = pd.read_csv(INPUT_PATH + 'trainv1.csv')
    train['target'] = train['target'].map(lambda x: data_log(x))
    submit = pd.read_csv(INPUT_PATH + 'submitv1.csv')
    return train, submit


def get_xgb_imp(xgb, feat_names):
    from numpy import array
    imp_vals = xgb.get_fscore()
    imp_dict = {feat_names[i]:float(imp_vals.get('f'+str(i),0.)) for i in range(len(feat_names))}
    total = array(imp_dict.values()).sum()
    return {k:v/total for k,v in imp_dict.items()}


def merge_dic(dicts):
    ret = {}
    for dict in dicts:
        for key in dict:
            val = dict[key]
            ret[key] = ret[key]+val if key in ret else val
    return ret

def data_log(x):
    if x <= 0:
        return 0
    else:
        return np.math.log(x + 1, 5)

In [146]:
print('begin to load data')
train, submit = load_data()

kfold = 10
skf = KFold(n_splits=kfold,shuffle=True, random_state=42)

begin to load data


In [147]:
params = {
        'objective': 'reg:linear',
        'max_depth': 6,
        'colsample_bytree': 0.8,
        'subsample': 0.8,
        'eta': 0.025,
        'gamma': 1,
        'reg_alpha': 0.5,
        'reg_lambda': 0.8,
        'eval_metric': 'rmse',
        'min_child_weight': 2,
        'silent': 1,
        'nthread': 6
    }



In [148]:
# cum = get_column_by_cum(train)
cum= ['comsume_counts_sum','comsume_amounts_sum','click_counts_sum']
#actives = [ 'limit_get_promoted','limit_get_promoted_ever']
#actives = ['dev_comsume_count', 'dev_consume_amount', 'dev_loan_amount', 'dev_loan_count', 'dev_plannum', 'dev_click_count']
#cum= []
actives =[]
all_features = [x for x in train.columns if not x in ['uid', 'target', 'active_date' ]+cum+actives]

X = train.drop(['uid', 'target', 'active_date']+cum+actives, axis=1).values
y = train.target.values

sub_id = submit.uid.values
to_submit = submit.drop(['uid', 'active_date']+cum+actives, axis=1)

sub = pd.DataFrame()
sub['uid'] = sub_id
sub['target'] = np.zeros_like(sub_id)

scores = []

importances = []

In [ ]:
for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    X_train, X_valid = X[train_index], X[test_index]
    y_train, y_valid = y[train_index], y[test_index]
    # Convert our data into XGBoost format
    d_train = xgb.DMatrix(X_train, y_train)
    d_valid = xgb.DMatrix(X_valid, y_valid)
    d_submit = xgb.DMatrix(to_submit.values)
    watchlist = [(d_train, 'train'), (d_valid, 'valid')]
    mdl = xgb.train(params, d_train, 4000, watchlist, early_stopping_rounds = 70, verbose_eval = 50)

    f_importance = get_xgb_imp(mdl,all_features)
    print(f_importance)
    importances.append(f_importance)

    score_this = np.sqrt(mean_squared_error(mdl.predict(d_valid, ntree_limit = mdl.best_ntree_limit), y_valid))
    print(score_this)
    scores.append(score_this)

    print('[Fold %d/%d Prediciton:]' % (i + 1, kfold))
    # Predict on our test data
    p_test = mdl.predict(d_submit, ntree_limit=mdl.best_ntree_limit)

    sub['target'] += p_test / kfold

[0]	train-rmse:2.43956	valid-rmse:2.4571
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 70 rounds.
[50]	train-rmse:1.8807	valid-rmse:1.90493
[100]	train-rmse:1.81442	valid-rmse:1.84797
[150]	train-rmse:1.79977	valid-rmse:1.84132
[200]	train-rmse:1.79117	valid-rmse:1.83983
[250]	train-rmse:1.785	valid-rmse:1.83944
[300]	train-rmse:1.77973	valid-rmse:1.8392
[350]	train-rmse:1.77459	valid-rmse:1.83916
Stopping. Best iteration:
[319]	train-rmse:1.77774	valid-rmse:1.83908

{'plannum': 0.050350116705568521, 'sex': 0.0033820797408660032, 'limit_get_promoted_ever': 0.0063830800743104842, 'loan_amount_cum': 0.06892773781736769, 'consume_amount': 0.058686228742914306, 'dev_loan_amount': 0.060067641594817318, 'limit_get_promoted': 0.0089553660744057535, 'loan_count': 0.045872433668365645, 'limit': 0.024913066260181964, 'consume_amount_cum': 0.05506597437240985, 'click_count_cum': 0.054637260039060637, 'comsume_

In [142]:
imp_sum = merge_dic(importances)
sort_rec = sorted(imp_sum.items(), key=lambda x:x[1])
print(sort_rec)

print('cv avg scores %s' % np.mean(scores))

[('active_year', 0.016768055683244545), ('sex', 0.036978126810906099), ('limit_get_promoted_ever', 0.064961582424497774), ('limit_get_promoted', 0.067634028538866728), ('age', 0.18494426109841872), ('limit', 0.25299770447169334), ('actived_months', 0.25346594062705197), ('loan_count_cum', 0.26028813773962972), ('active_month', 0.29881216614508732), ('plannum_cum', 0.38216748615924911), ('loan_count', 0.38481453703125779), ('comsume_count', 0.4153244102204014), ('plannum', 0.43664416244400833), ('dev_consume_amount_cum', 0.53141464084429635), ('consume_amount_cum', 0.53716529684938596), ('comsume_count_cum', 0.53742270789377866), ('loan_amount_cum', 0.58365126235101461), ('dev_loan_amount', 0.59433319010893071), ('click_count_cum', 0.59467169110051454), ('loan_amount', 0.61717853892249164), ('consume_amount', 0.62922990895398978), ('click_count', 0.63566495272090662), ('dev_click_count_cum', 0.67827532265426294), ('dev_loan_amount_cum', 1.005191888206115)]
cv avg scores 1.84540806232


In [82]:
scores

[1.8412667098272195,
 1.8495803893666833,
 1.8270694805331991,
 1.8290116761340895,
 1.8451969781698268,
 1.8260666156041556,
 1.8017041532580538,
 1.8159043790591873,
 1.7847619087813988,
 1.823307141082499]

In [143]:
from datetime import datetime
sub.loc[sub.target < 0, 'target'] = 0
sub.to_csv(OUTPUT_PATH +"sub{}.csv".format(datetime.now().strftime('%Y%m%d_%H%M%S')), index=False)

In [326]:
all_features = [x for x in train.columns if not x in ['uid', 'target', 'active_date']]

X = train.drop(['uid', 'target', 'active_date'], axis=1)
y = train.target

In [348]:
xgb_params = {
        'objective': ['reg:linear'],
        'learning_rate': [0.025],
        'max_depth': [6],
        'min_child_weight': [2],
        'silent': [1],
        'subsample': [ 0.8],
        'colsample_bytree': [0.8],
        'n_estimators': [400],
        'gamma': [1,3],
        'reg_alpha': [0.5,0.8],
        'reg_lambda': [0.8,1.3] 
    }

In [349]:

from sklearn.model_selection import GridSearchCV

import xgboost as xgb


In [350]:
modelXgboost = xgb.XGBRegressor()

    # Run the grid search
grid_obj_xgb = GridSearchCV(modelXgboost, xgb_params, cv=KFold(n_splits=5, shuffle=True, random_state=42), n_jobs=2, verbose=2, refit=True,scoring='mean_squared_error')
grid_obj_xgb = grid_obj_xgb.fit(X, y)


Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] reg_alpha=0.5, colsample_bytree=0.8, silent=1, learning_rate=0.025, min_child_weight=2, n_estimators=400, subsample=0.8, reg_lambda=0.8, objective=reg:linear, max_depth=6, gamma=1 
[CV] reg_alpha=0.5, colsample_bytree=0.8, silent=1, learning_rate=0.025, min_child_weight=2, n_estimators=400, subsample=0.8, reg_lambda=0.8, objective=reg:linear, max_depth=6, gamma=1 


/Library/Python/2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Library/Python/2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Library/Python/2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Library/Python/2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)


[CV]  reg_alpha=0.5, colsample_bytree=0.8, silent=1, learning_rate=0.025, min_child_weight=2, n_estimators=400, subsample=0.8, reg_lambda=0.8, objective=reg:linear, max_depth=6, gamma=1, total= 1.4min
[CV] reg_alpha=0.5, colsample_bytree=0.8, silent=1, learning_rate=0.025, min_child_weight=2, n_estimators=400, subsample=0.8, reg_lambda=0.8, objective=reg:linear, max_depth=6, gamma=1 
[CV]  reg_alpha=0.5, colsample_bytree=0.8, silent=1, learning_rate=0.025, min_child_weight=2, n_estimators=400, subsample=0.8, reg_lambda=0.8, objective=reg:linear, max_depth=6, gamma=1, total= 1.4min
[CV] reg_alpha=0.5, colsample_bytree=0.8, silent=1, learning_rate=0.025, min_child_weight=2, n_estimators=400, subsample=0.8, reg_lambda=0.8, objective=reg:linear, max_depth=6, gamma=1 


/Library/Python/2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Library/Python/2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Library/Python/2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Library/Python/2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)


[CV]  reg_alpha=0.5, colsample_bytree=0.8, silent=1, learning_rate=0.025, min_child_weight=2, n_estimators=400, subsample=0.8, reg_lambda=0.8, objective=reg:linear, max_depth=6, gamma=1, total= 1.5min
[CV] reg_alpha=0.5, colsample_bytree=0.8, silent=1, learning_rate=0.025, min_child_weight=2, n_estimators=400, subsample=0.8, reg_lambda=0.8, objective=reg:linear, max_depth=6, gamma=1 
[CV]  reg_alpha=0.5, colsample_bytree=0.8, silent=1, learning_rate=0.025, min_child_weight=2, n_estimators=400, subsample=0.8, reg_lambda=0.8, objective=reg:linear, max_depth=6, gamma=1, total= 1.5min
[CV] reg_alpha=0.5, colsample_bytree=0.8, silent=1, learning_rate=0.025, min_child_weight=2, n_estimators=400, subsample=0.8, reg_lambda=1.3, objective=reg:linear, max_depth=6, gamma=1 


...

[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed: 28.4min


[CV]  reg_alpha=0.8, colsample_bytree=0.8, silent=1, learning_rate=0.025, min_child_weight=2, n_estimators=400, subsample=0.8, reg_lambda=1.3, objective=reg:linear, max_depth=6, gamma=3, total= 1.4min
[CV] reg_alpha=0.8, colsample_bytree=0.8, silent=1, learning_rate=0.025, min_child_weight=2, n_estimators=400, subsample=0.8, reg_lambda=1.3, objective=reg:linear, max_depth=6, gamma=3 


/Library/Python/2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Library/Python/2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Library/Python/2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Library/Python/2.7/site-packages/sklearn/metrics/scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)


[CV]  reg_alpha=0.8, colsample_bytree=0.8, silent=1, learning_rate=0.025, min_child_weight=2, n_estimators=400, subsample=0.8, reg_lambda=1.3, objective=reg:linear, max_depth=6, gamma=3, total= 1.2min
[CV]  reg_alpha=0.8, colsample_bytree=0.8, silent=1, learning_rate=0.025, min_child_weight=2, n_estimators=400, subsample=0.8, reg_lambda=1.3, objective=reg:linear, max_depth=6, gamma=3, total= 1.2min


[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed: 29.7min finished


In [351]:
clf = grid_obj_xgb.best_estimator_
grid_obj_xgb.best_params_

{'colsample_bytree': 0.8,
 'gamma': 1,
 'learning_rate': 0.025,
 'max_depth': 6,
 'min_child_weight': 2,
 'n_estimators': 400,
 'objective': 'reg:linear',
 'reg_alpha': 0.5,
 'reg_lambda': 0.8,
 'silent': 1,
 'subsample': 0.8}

In [59]:

users = pd.read_csv(INPUT_PATH + 't_user.csv')
orders = pd.read_csv(INPUT_PATH + 't_order.csv')
loans = pd.read_csv(INPUT_PATH + 't_loan.csv')
loans_sum = pd.read_csv(INPUT_PATH + 't_loan_sum.csv')
clicks = pd.read_csv(INPUT_PATH + 't_click.csv')

In [64]:
loans.loan_amount = 5**loans.loan_amount
loans_sum.loan_sum = 5 ** loans_sum.loan_sum - 1
orders.price = 5 ** orders.price
orders.discount = 5 ** orders.discount - 1
users.limit = 5 ** users.limit -1


In [65]:
orders["consume_amount"] = orders["price"] * orders["qty"] - orders["discount"]

In [68]:
import matplotlib.pyplot as plt

In [69]:
plt.figure(figsize=(8,6))
plt.scatter(range(orders.shape[0]), np.sort(orders.consume_amount.values))
plt.xlabel('index', fontsize=12)
plt.ylabel('consume_amount', fontsize=12)
plt.show()

/System/Library/Frameworks/Python.framework/Versions/2.7/Extras/lib/python/matplotlib/collections.py:548: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == 'face':


In [91]:
ulimit = np.percentile(orders.consume_amount.values, 99.9)
llimit = 0


In [81]:
orders.consume_amount.isnull().values.any()

False

In [94]:
orders = orders[ orders.consume_amount > llimit ]
orders = orders[ orders.consume_amount < ulimit ]

In [80]:
orders.consume_amount = orders.consume_amount.fillna(0)

In [79]:
orders[orders.cate_id==33].price.describe()

count    395972.000000
mean        999.471488
std        1802.303636
min           1.000000
25%          30.000000
50%          89.000000
75%        1100.000000
max       69888.999996
Name: price, dtype: float64

In [92]:
ulimit

44036.000002161658

In [93]:
llimit

0

In [96]:
orders.shape

(5384276, 7)

In [97]:
5400778-5384276

16502

In [113]:
user_info = pd.read_csv(OUTPUT_PATH + 'user_info.csv')

In [119]:
for c in user_info.columns:
        user_info[c] = user_info[c].fillna(0)

In [122]:
for month in [8, 9, 10, 11, 12]:
        user_info = cummulate_data_by_month(user_info, month)

In [121]:
CC = '{}_comsume_count'
CM = '{}_consume_amount'
LM = '{}_loan_amount'
LC = '{}_loan_count'
CKC = '{}_click_count'
CUM = '_cum'
basic = ['uid', 'age', 'sex', 'active_date', 'limit']
TARGET = 'target'

In [123]:
list(user_info)

['uid',
 'age',
 'sex',
 'active_date',
 'limit',
 '8_comsume_count',
 '9_comsume_count',
 '10_comsume_count',
 '11_comsume_count',
 '8_consume_amount',
 '9_consume_amount',
 '10_consume_amount',
 '11_consume_amount',
 '8_loan_amount',
 '9_loan_amount',
 '10_loan_amount',
 '11_loan_amount',
 '8_loan_count',
 '9_loan_count',
 '10_loan_count',
 '11_loan_count',
 '8_click_count',
 '9_click_count',
 '10_click_count',
 '11_click_count',
 '8_comsume_count_cum',
 '8_consume_amount_cum',
 '8_loan_amount_cum',
 '8_loan_count_cum',
 '8_click_count_cum',
 '9_comsume_count_cum',
 '9_consume_amount_cum',
 '9_loan_amount_cum',
 '9_loan_count_cum',
 '9_click_count_cum',
 '10_comsume_count_cum',
 '10_consume_amount_cum',
 '10_loan_amount_cum',
 '10_loan_count_cum',
 '10_click_count_cum',
 '11_comsume_count_cum',
 '11_consume_amount_cum',
 '11_loan_amount_cum',
 '11_loan_count_cum',
 '11_click_count_cum',
 '12_comsume_count_cum',
 '12_consume_amount_cum',
 '12_loan_amount_cum',
 '12_loan_count_cum',
 '

In [149]:
all_features

['age',
 'sex',
 'limit',
 'active_month',
 'active_year',
 'comsume_count',
 'consume_amount',
 'loan_amount',
 'loan_count',
 'plannum',
 'click_count',
 'limit_get_promoted',
 'limit_get_promoted_ever',
 'comsume_count_cum',
 'consume_amount_cum',
 'loan_amount_cum',
 'loan_count_cum',
 'click_count_cum',
 'plannum_cum',
 'actived_months',
 'dev_consume_amount_cum',
 'dev_loan_amount_cum',
 'dev_loan_amount',
 'dev_click_count_cum']

In [43]:
train.head()

,uid,age,sex,active_date,limit,active_month,active_year,comsume_count,consume_amount,loan_amount,...,consume_amount_cum,loan_amount_cum,loan_count_cum,click_count_cum,plannum_cum,actived_months,comsume_counts_sum,comsume_amounts_sum,click_counts_sum,target
0,26308,30,1,2016-02-16,15000.000001,2,1,1.0,700.000000,2500.0,...,0.0,0.0,0.0,0.0,0.0,6,3.0,1283.000000,213.0,5.070593
1,78209,40,1,2016-02-21,5000.000000,2,1,38.0,8429.100000,0.0,...,0.0,0.0,0.0,0.0,0.0,6,86.0,12510.600000,0.0,0.000000
2,51930,35,1,2016-04-19,24999.999999,4,1,2.0,790.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,4,3.0,830.900000,35.0,6.405333
3,10113,25,1,2016-03-12,24999.999999,3,1,23.0,2251.800000,0.0,...,0.0,0.0,0.0,0.0,0.0,5,34.0,2775.000000,154.0,0.000000
4,17067,35,1,2016-02-16,15000.000001,2,1,51.0,19244.299999,0.0,...,0.0,0.0,0.0,0.0,0.0,6,148.0,33788.489999,62.0,0.000000


In [59]:
list(train)

['uid',
 'age',
 'sex',
 'active_date',
 'limit',
 'active_month',
 'active_year',
 'comsume_count',
 'consume_amount',
 'loan_amount',
 'loan_count',
 'plannum',
 'click_count',
 'comsume_count_cum',
 'consume_amount_cum',
 'loan_amount_cum',
 'loan_count_cum',
 'click_count_cum',
 'plannum_cum',
 'actived_months',
 'comsume_counts_sum',
 'comsume_amounts_sum',
 'click_counts_sum',
 'target']

In [50]:
train[train['uid']==26308]

,uid,age,sex,active_date,limit,active_month,active_year,comsume_count,consume_amount,loan_amount,...,consume_amount_cum,loan_amount_cum,loan_count_cum,click_count_cum,plannum_cum,actived_months,comsume_counts_sum,comsume_amounts_sum,click_counts_sum,target
0,26308,30,1,2016-02-16,15000.000001,2,1,1.0,700.0,2500.000000,...,0.0,0.0,0.0,0.0,0.0,6,3.0,1283.0,213.0,5.070593
90993,26308,30,1,2016-02-16,15000.000001,2,1,0.0,0.0,3500.000000,...,700.0,2502.0,2.0,55.0,2.0,7,3.0,1283.0,213.0,6.033893
181986,26308,30,1,2016-02-16,15000.000001,2,1,2.0,583.0,16500.000001,...,700.0,6004.0,4.0,87.0,4.0,8,3.0,1283.0,213.0,5.501180


In [112]:
all_features

['age',
 'sex',
 'limit',
 'active_month',
 'active_year',
 'comsume_count',
 'consume_amount',
 'loan_amount',
 'loan_count',
 'plannum',
 'click_count',
 'limit_get_promoted',
 'limit_get_promoted_ever',
 'comsume_count_cum',
 'consume_amount_cum',
 'loan_amount_cum',
 'loan_count_cum',
 'click_count_cum',
 'plannum_cum',
 'actived_months']

In [24]:
subs = pd.read_csv(OUTPUT_PATH + 'submission20171117_164203planWithActiveDate.csv', header=None)

In [48]:
# ['comsume_counts_sum','comsume_amounts_sum','click_counts_sum']
submit['click_counts_sum'].describe()

count    90993.000000
mean       120.152275
std        185.058858
min          0.000000
25%         16.000000
50%         64.000000
75%        152.000000
max       5092.000000
Name: click_counts_sum, dtype: float64

In [40]:
submit['click_counts_sum'].describe()

count    272979.000000
mean        120.152275
std         185.058180
min           0.000000
25%          16.000000
50%          64.000000
75%         152.000000
max        5092.000000
Name: click_counts_sum, dtype: float64

In [79]:
sub.describe()

,uid,target
count,90993.000000,90993.000000
mean,45497.000000,1.284766
std,26267.560859,1.516372
min,1.000000,-0.058015
25%,22749.000000,0.280137
50%,45497.000000,0.457074
75%,68245.000000,1.941764
max,90993.000000,7.410984


In [80]:
sub[sub.target<0].shape

(1, 2)

In [150]:
train.shape

(181986, 30)